In [1]:
import pandas as pd
import os

# Charger les données collectées
folder = 'pollution_data'
file_list = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.csv')]

# Combiner tous les fichiers CSV en un seul DataFrame
df_list = [pd.read_csv(file) for file in file_list]
df = pd.concat(df_list, ignore_index=True)

# Vérification des valeurs manquantes
if df.isnull().values.any():
    print("Des valeurs manquantes ont été détectées.")
    # Gérer les valeurs manquantes - par exemple, les remplacer par la moyenne de la colonne
    df.fillna(df.mean(), inplace=True)

# Vérification des types de données et conversion si nécessaire
df['datetime'] = pd.to_datetime(df['datetime'])

# Vérification de la cohérence des données
# Assurer que les concentrations de polluants sont dans des plages raisonnables
pollutant_columns = ['pm2_5', 'pm10', 'o3', 'no2', 'so2', 'co']
for column in pollutant_columns:
    if df[column].min() < 0:
        print(f"Valeur négative détectée dans {column}.")
    # Fixer les valeurs négatives à 0 si nécessaire
    df[column] = df[column].clip(lower=0)

# Transformation : Agrégation des données par jour (moyenne journalière)
df['date'] = df['datetime'].dt.date
daily_df = df.groupby('date').agg({
    'aqi': 'mean',
    'pm2_5': 'mean',
    'pm10': 'mean',
    'o3': 'mean',
    'no2': 'mean',
    'so2': 'mean',
    'co': 'mean'
}).reset_index()

# Sauvegarder le DataFrame nettoyé et transformé
daily_df.to_csv('pollution_data_cleaned.csv', index=False)
print("Données nettoyées et transformées sauvegardées dans 'pollution_data_cleaned.csv'")

Données nettoyées et transformées sauvegardées dans 'pollution_data_cleaned.csv'
